In [ ]:
# import required libraries
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import pandas as pd
import string

# download punctuation and stopwords from nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 https://develo

In [7]:
# load tweets_df and view
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()
from pyspark import SparkFiles
url ="https://tweet-2022.s3.amazonaws.com/Tweets.csv"

In [15]:
# optimized_df = spark_tweets_df.toPandas()
optimized_df = pd.read_csv("Tweets.csv")
optimized_df=optimized_df.loc[optimized_df['sentiment']!="neutral"]
# optimized_df=optimized_df.dropna()
optimized_df

,textID,text,selected_text,sentiment
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
...,...,...,...,...
27475,b78ec00df5,enjoy ur night,enjoy,positive
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [16]:
# get dataframe ready for processing

# make sure the tweets in column "text" are strings
optimized_df['text'] = optimized_df['text'].astype('str')

# delete the unneccessary columns
optimized_df = optimized_df.drop(columns=["textID", "selected_text"])
optimized_df=optimized_df.rename(columns={'sentiment':'class'})
optimized_df=optimized_df[['class','text']]

In [17]:
def nb_process_tweets(tweet):
    # make the text all lowercase
    tweet = tweet.lower()
    
    # remove punctuation
    tweet = "".join(char for char in tweet if char not in string.punctuation)
  
    # remove urls
    tweet_wo_stop = "".join([i for i in tweet if 'http' not in i])
    
    
    # lemmatization
    lemm = WordNetLemmatizer()
    lemmed = [lemm.lemmatize(word) for word in tweet_wo_stop]
    
    # put string together
    final_tweet = "".join(lemmed)
    
    return final_tweet

In [18]:
optimized_df['text'] = optimized_df['text'].apply(lambda x: nb_process_tweets(x))
optimized_df['length']=""
for index,row in optimized_df.iterrows():
  tweet_length=len(row['text'])
  optimized_df.loc[index,'length']=tweet_length
optimized_df

,class,text,length
1,negative,sooo sad i will miss you here in san diego,43
2,negative,my boss is bullying me,22
3,negative,what interview leave me alone,30
4,negative,sons of why couldnt they put them on the rel...,69
6,positive,2am feedings for the baby are fun when he is a...,64
...,...,...,...
27475,positive,enjoy ur night,16
27476,negative,wish we could come see u on denver husband l...,76
27477,negative,ive wondered about rake to the client has ma...,115
27478,positive,yay good for both of you enjoy the break you...,109


In [37]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [38]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [39]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])


In [41]:
from pyspark.sql import SQLContext
sc = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)
opt_spark_tweets = sqlContext.createDataFrame(optimized_df)
cleaner = data_prep_pipeline.fit(opt_spark_tweets)
cleaned = cleaner.transform(opt_spark_tweets)

opt_spark_tweets = sqlContext.createDataFrame(optimized_df)
opt_cleaner = data_prep_pipeline.fit(opt_spark_tweets)
opt_cleaned = opt_cleaner.transform(opt_spark_tweets)

/content/spark-3.2.3-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [42]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
opt_training, opt_testing = opt_cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
opt_nb = NaiveBayes(labelCol="label", featuresCol="features", modelType="multinomial")
opt_predictor = opt_nb.fit(opt_training)

In [43]:
opt_test_results = opt_predictor.transform(opt_testing)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

opt_acc_eval = MulticlassClassificationEvaluator(labelCol="label")
opt_acc = opt_acc_eval.evaluate(opt_test_results)
print("Accuracy of model at predicting reviews was: %f" % opt_acc)

Accuracy of model at predicting reviews was: 0.805526
